In [1]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim

In [2]:
import torch.utils.data
import torch.utils.data.distributed

In [ ]:
#import torchvision.transforms as transforms

In [ ]:
#import torchvision.datasets as datasets
#import torchvision.models

In [3]:
from torch.autograd import Variable
from tqdm import tqdm
import os
from PIL import Image
import cv2

In [4]:
from torch.optim.lr_scheduler import StepLR
#from setting import parse_opts


In [5]:
from setting_CT_test import parse_opts

In [6]:
from CT_preprocessing import CTDataset

In [38]:
import logging

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.DEBUG)

log = logging.getLogger()

In [36]:
from utils import logger

ImportError: cannot import name 'logger' from 'utils' (C:\Users\ASUS\.conda\envs\py370\lib\site-packages\utils\__init__.py)

In [7]:
from model import generate_model
import torch
import numpy as np
from torch import nn
from torch import optim
from torch.optim import lr_scheduler, optimizer
from torch.utils.data import DataLoader
import time
#from utils.logger import log
from scipy import ndimage
import os
import matplotlib.pyplot as plt
#from test_CT import test

import torch.nn.functional as F
import matplotlib.pyplot as plt

In [8]:
#判断环境是CPU运行还是GPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [ ]:
#DEVICE=torch.device('cuda:0')

In [ ]:
#DEVICE

In [9]:
sets = parse_opts()

In [10]:
if sets.ci_test:   
        sets.img_list_test = 'D:/MedicalNet/MedicalNet/data/test.txt'
        sets.n_epochs_test = 1
        sets.data_root_test = './data'
        training_dataset = CTDataset(sets.data_root, sets.img_list, sets, phase='train')
        test_dataset = CTDataset(sets.data_root_test, sets.img_list_test, sets, phase='test')

Processing 7 datas
Processing 2 datas


In [11]:
if sets.no_cuda:
    sets.pin_memory = False
else:
    sets.pin_memory = True
    
sets.num_workers_test = 0
sets.input_D_test = 50       # 建议放到上面去增加代码的可读性
sets.input_H_test = 224
sets.input_W_test = 224
data_loader = DataLoader(training_dataset, batch_size=sets.batch_size, shuffle=True, num_workers=sets.num_workers,
                             pin_memory=sets.pin_memory)
test_data_loader = DataLoader(test_dataset, batch_size=sets.batch_size, shuffle=False,
                                  num_workers=sets.num_workers_test, pin_memory=sets.pin_memory)

In [13]:
print(len(data_loader))

7


In [ ]:
for i, data in enumerate(test_data_loader):
	# i表示第几个batch， data表示该batch对应的数据，包含data和对应的labels
    print("第 {} 个Batch \n{}".format(i, data))

In [14]:
# 删除隐藏文件/文件夹
for root, dirs, files in os.walk('D:/MedicalNet/MedicalNet/datasets'):
    for file in files:
        if 'ipynb_checkpoints' in file:
            os.remove(os.path.join(root, file))
    if 'ipynb_checkpoints' in root:
        shutil.rmtree(root)

In [15]:
torch.manual_seed(sets.manual_seed)
model, parameters = generate_model(sets)
print(model)

loading pretrained model D:/MedicalNet/MedicalNet/pretrain/MedicalNet_pytorch_files/pretrain/resnet_50.pth
ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_

In [16]:
criterion = nn.CrossEntropyLoss() #交叉熵损失函数
num_ftrs = model.fc.in_features

In [17]:
# model.fc = nn.Linear(num_ftrs, 10)
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 224),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(224, 4),
    nn.LogSoftmax(dim=1)
)

params = [
    {'params': parameters['base_parameters'], 'lr': sets.learning_rate},
    {'params': parameters['new_parameters'], 'lr': sets.learning_rate * 100}
    ]

In [18]:
device=torch.device('cuda')#注意一下这个跑了之后似乎网络有一点变化
model.to(DEVICE)

ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv3d(64

In [19]:
# 选择简单暴力的Adam优化器，学习率调低
optimizer = torch.optim.SGD(params, momentum=0.9, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

In [20]:
def adjust_learning_rate(optimizer, epoch):
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

In [42]:
#def train(model, device, train_loader, optimizer, epoch):
def train(data_loader, model,optimizer,scheduler,total_epochs,save_interval, save_folder, sets):
    batches_per_epoch = len(data_loader)
    log.info('{} epochs in total, {} batches per epoch'.format(total_epochs, batches_per_epoch))
    # loss_class = nn.CrossEntropyLoss()
    print("Current setting is:")
    print(sets)
    print("\n\n") 
    model.train()
    train_time_sp = time.time()
    sum_loss = 0
    train_acc = 0
    total_num = len(data_loader.dataset)#注意一下会不会报错
    print(total_num, len(data_loader))

       
    log.info('Start epoch {}'.format(epoch)) 
    scheduler.step()
    log.info('lr = {}'.format(scheduler.get_lr()))

    for batch_id, batch_data in enumerate(data_loader):
        batch_id_sp = epoch * batches_per_epoch
        volumes, labels = batch_data
        if not sets.no_cuda:
            volumes = volumes.cuda()
            labels = labels.cuda().long()
        #data, target = Variable(data).to(device), Variable(target).to(device)
        im = Variable(volumes)
        output = model(volumes)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss

        out_t = output.argmax(dim=1) #取出预测的最大值
        num_correct = (out_t == labels).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc

        avg_batch_time = (time.time() - train_time_sp) / (1 + batch_id_sp)
        log.info(
            'Batch: {}-{} ({}), loss = {:.3f}, avg_batch_time = {:.3f}' \
                .format(epoch, batch_id, batch_id_sp, loss.item(), avg_batch_time))

        if not sets.ci_test:
            # save model
            if batch_id == 0 and batch_id_sp != 0 and batch_id_sp % save_interval == 0:
            #if batch_id_sp != 0 and batch_id_sp % save_interval == 0:
                model_save_path = '{}_epoch_{}_batch_{}.pth.tar'.format(save_folder, epoch, batch_id)
                model_save_dir = os.path.dirname(model_save_path)
                if not os.path.exists(model_save_dir):
                    os.makedirs(model_save_dir)

                log.info('Save checkpoints: epoch = {}, batch_id = {}'.format(epoch, batch_id)) 
                torch.save({
                            'ecpoch': epoch,
                            'batch_id': batch_id,
                            'state_dict': model.state_dict(),
                            'optimizer': optimizer.state_dict()},
                            model_save_path)
        if sets.ci_test:
            exit()
            
        if (batch_id + 1) % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_id + 1) * len(volumes), len(data_loader.dataset),
                       100. * (batch_id + 1) / len(data_loader), loss.item()))
            
    ave_loss = sum_loss / len(train_loader)
    ave_acc = train_acc / len(train_loader)
    print('epoch:{}, train_acc: {}, loss:{}'.format(epoch, ave_acc, ave_loss))
    return ave_acc, ave_loss

In [26]:
# 定义训练过程

def val(test_data_loader, model, sets):
    model.eval()
    sets.phase = 'test'#
    checkpoint = torch.load(sets.resume_path)#
    test_loss = 0
    correct = 0
    total_num = len(test_data_loader.test_dataset)
    # print(total_num, len(test_loader))
    with torch.no_grad():
        for batch_id, batch_data in enumerate(test_data_loader):
            #data, target = Variable(data).to(device), Variable(target).to(device)
            volumes, label = batch_data
            if not sets.no_cuda:
                volumes = volumes.cuda()
                label = label.cuda().long()
            output = model(volumes)
            output.load_state_dict(checkpoint['state_dict'])#
            loss = criterion(output,label)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == label)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_data_loader)
        print('Val set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))
        

    return acc, avgloss


In [ ]:
for i in range(5):
    sets = parse_opts()
    sets.img_list_test = 'D:/MedicalNet/MedicalNet/data/Image_NII/test_fold[i].txt'
    sets.n_epochs_test = 1#这是啥
    sets.data_root_test = 'D:/MedicalNet/MedicalNet/data'
    sets.num_workers_test = 0
    sets.input_D_test = 50       # 设置输入Size
    sets.input_H_test = 224
    sets.input_W_test = 224
        
    torch.manual_seed(sets.manual_seed)
    model, parameters = generate_model(sets)
    print(model)
        

In [43]:
#在此处增加五折
train_acc_list, train_loss_list, val_acc_list, val_loss_list = [], [], [], []
total_epochs=sets.n_epochs
modellr=sets.learning_rate
for epoch in range(1, total_epochs + 1):
    adjust_learning_rate(optimizer, epoch)
    train_acc, train_loss = train(data_loader,model, optimizer, scheduler,total_epochs, save_interval=sets.save_intervals, 
                                  save_folder=sets.save_folder, sets=sets)
    val_acc, val_loss = val(test_data_loader, model, sets)
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)


2023-05-15 21:59:13 INFO     [3071507603.py:4] 20 epochs in total, 7 batches per epoch
2023-05-15 21:59:13 INFO     [3071507603.py:17] Start epoch 1
2023-05-15 21:59:13 INFO     [3071507603.py:19] lr = [0.0009801, 0.0009801]


lr: 0.001
Current setting is:
Namespace(batch_size=1, ci_test=True, data_root='D:/MedicalNet/MedicalNet/data', data_root_test='./data', gpu_id=0, img_list='D:/MedicalNet/MedicalNet/data/train.txt', img_list_test='D:/MedicalNet/MedicalNet/data/test.txt', input_D=50, input_D_test=50, input_H=224, input_H_test=224, input_W=224, input_W_test=224, learning_rate=0.001, manual_seed=1, model='resnet', model_depth=50, n_cls_classes=4, n_epochs=20, n_epochs_test=1, new_layer_names=['conv_seg'], no_cuda=True, num_workers=4, num_workers_test=0, phase='train', pin_memory=False, pretrain_path='D:/MedicalNet/MedicalNet/pretrain/MedicalNet_pytorch_files/pretrain/resnet_50.pth', resnet_shortcut='B', resume_path='', save_folder='D:/MedicalNet/MedicalNet/trails/models/resnet_50', save_intervals=1)



7 7


2023-05-15 21:59:17 INFO     [3071507603.py:45] Batch: 1-0 (7), loss = 1.369, avg_batch_time = 0.558
2023-05-15 21:59:20 INFO     [3071507603.py:45] Batch: 1-1 (7), loss = 1.546, avg_batch_time = 0.859
2023-05-15 21:59:22 INFO     [3071507603.py:45] Batch: 1-2 (7), loss = 1.789, avg_batch_time = 1.171
2023-05-15 21:59:25 INFO     [3071507603.py:45] Batch: 1-3 (7), loss = 1.413, avg_batch_time = 1.471
2023-05-15 21:59:27 INFO     [3071507603.py:45] Batch: 1-4 (7), loss = 0.932, avg_batch_time = 1.785
2023-05-15 21:59:29 INFO     [3071507603.py:45] Batch: 1-5 (7), loss = 1.050, avg_batch_time = 2.085
2023-05-15 21:59:32 INFO     [3071507603.py:45] Batch: 1-6 (7), loss = 1.686, avg_batch_time = 2.394


NameError: name 'train_loader' is not defined

In [2]:
import matplotlib.pyplot as plt

epochs_range = range(total_epochs)
print(epochs_range, train_acc_list)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_acc_list, label='Training Accuracy')
plt.plot(epochs_range, val_acc_list, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_loss_list, label='Training Loss')
plt.plot(epochs_range, val_loss_list, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./acc-loss.jpg')

NameError: name 'total_epochs' is not defined

In [ ]:
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from pathlib import Path

classes = ('cat', 'dog')
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),      
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load("model.pth")
model.eval()
model.to(DEVICE)

dataset_test = datasets.ImageFolder('猫狗dataset/test', transform_test)
print(len(dataset_test))
# 对应文件夹的label

y_true, y_sore = [], []
for index in range(len(dataset_test)):
    item = dataset_test[index]
    img, label = item
    img.unsqueeze_(0)
    data = Variable(img).to(DEVICE)
    output = model(data)
    _, pred = torch.max(output.data, 1)
    y_true.append(label)
    y_sore.append(pred.data.item())
    print('Image Name:{}, label:{}, predict:{}'.format(dataset_test.imgs[index][0], classes[label], classes[pred.data.item()]))
    index += 1
